In [19]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, preprocessing, model_selection
from sklearn.metrics import mean_squared_error

In [29]:
def load_dataset():
    x, y = datasets.load_iris(return_X_y=True)
    x = preprocessing.scale(x)
    y = y.reshape(len(y), 1)
    return x, y

In [35]:
def test():
    #Data loading
    x, y = load_dataset()
    x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size= 0.4, random_state=0)

    print('Data loaded')
    print('Train = ', x_train.shape[0])
    print('Test  = ', x_test.shape[0])
    print('Cols  = ', x_train .shape[1])

test()

Data loaded
Train =  90
Test  =  60
Cols  =  4
[[1]
 [0]
 [2]
 [1]
 [1]
 [1]
 [1]
 [2]
 [0]
 [0]
 [2]
 [1]
 [0]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [2]
 [1]
 [0]
 [2]
 [2]
 [2]
 [2]
 [0]
 [0]
 [2]
 [2]
 [0]
 [2]
 [0]
 [2]
 [2]
 [0]
 [0]
 [2]
 [0]
 [0]
 [0]
 [1]
 [2]
 [2]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [2]
 [1]
 [2]
 [1]
 [0]
 [2]
 [0]
 [2]
 [0]
 [0]
 [2]
 [0]
 [2]
 [1]
 [1]
 [1]
 [2]
 [2]
 [1]
 [1]
 [0]
 [1]
 [2]
 [2]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [2]
 [1]
 [2]
 [0]]
